In [1]:
# importações
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from modelos.CONSTANTES import *

In [2]:
# teste:
for rede in redes_sociais.keys():

    ######################################
    # lidando com a rede social
    ######################################
    print('Rede Social --', rede)

    G = nx.read_edgelist("redes/" + redes_sociais[rede], comments='%', nodetype=int)
    print('<k>:', np.array(G.degree)[:, 1].mean(), '\tN:', len(G))

Rede Social -- youtube
<k>: 5.265045951590022 	N: 1134890
